In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
     
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2024-08-07 18:09:20.234371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 18:09:20.234529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 18:09:20.370441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

/tmp/ipykernel_25/2653809078.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [3]:
df_2018 = pd.read_csv('/kaggle/input/isic-2018-jpg-224x224-resized/train-metadata.csv')
df_2019 = pd.read_csv('/kaggle/input/isic-2019-jpg-224x224-resized/train-metadata.csv')
df_2020 = pd.read_csv('/kaggle/input/isic-2020-jpg-224x224-resized/train-metadata.csv')

In [4]:
df_2018.head()

,Unnamed: 0,isic_id,patient_id,target
0,0,ISIC_0028498,HAM_0000000,0.0
1,1,ISIC_0027859,HAM_0000001,0.0
2,2,ISIC_0034246,HAM_0000002,1.0
3,3,ISIC_0027886,HAM_0000003,0.0
4,4,ISIC_0024645,HAM_0000004,0.0


In [5]:
df.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.00000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.00000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.00000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.00000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.00000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.00000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.51452,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.66469,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.00000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.00000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


## Constants

In [6]:
RANDOM_STATE = 42
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 32
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
img_augmentation_layers = [
    layers.RandomRotation(factor=0.15),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
]




## Helper functions

In [7]:
import tensorflow as tf
from typing import NamedTuple, Optional
from sklearn import preprocessing 


def add_path(path,dir):
    return f"{dir}/{path}.jpg"



def get_sample(df,ratio):
    df_positive = df[df['target']==1].reset_index(drop=True)
    df_negative = df[df['target']==0].reset_index(drop=True)
    df_negative= resample(df_negative,n_samples=len(df_positive)*ratio,random_state=RANDOM_STATE)
#     df_positive = resample(df_positive,n_samples=len(df_positive),random_state=RANDOM_STATE)
    
    df_concat = pd.concat([df_positive,df_negative])
    df_concat = shuffle(df_concat).reset_index(drop=True)
    
    print(f'positive shape: {df_positive.shape}, negative shape: {df_negative.shape}')
    print(f'Full dataset shape: {df_concat.shape}')
    
    return df_concat

def visualize(df,idx):
    images = []
    labels = []
    for id in idx:
        current_row = df.iloc[id]
        images.append(plt.imread(current_row["file_path"]))
        labels.append(current_row["target"])
    
    for i, (image, label) in enumerate(zip(images, labels), start=1):
        plt.subplot(3, 3, i)  # Adjusted to start from 1
        plt.imshow(image)
        plt.axis("off")
        plt.title(label)
    plt.show()

def encode(df, columns):
    label_encoder = preprocessing.LabelEncoder() 
    for col in columns:
        df[col]= label_encoder.fit_transform(df[col])
    return df

def scale_columns(df, columns):
    scaler = preprocessing.StandardScaler()
    
    # Transform each specified column
    for col in columns:
        df[col] = scaler.fit_transform(df[[col]])
        
    return df

class Metadata(NamedTuple):
    image: Optional[tf.Tensor] = None
    metadata: Optional[tf.Tensor] = None



def img_augmentation(images):
    for layer in img_augmentation_layers:
        images = layer(images)
    return images

def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = image / 255.0
    image = img_augmentation(image)
    return image

def preprocess_metadata(filtered_dict):
    # Convert metadata to tensor
    tensor_dict = tf.stack(list(filtered_dict.values()), axis=-1)
    return tensor_dict

def load_image_and_metadata(metadata, image_size):
    image = preprocess_image(metadata['file_path'], image_size)
    filtered_dict = {k: v for k, v in metadata.items() if k != 'file_path'}
    metadata_tensor = preprocess_metadata(filtered_dict)
    return Metadata(image=image, metadata=metadata_tensor)


def unbatch_test_dataset(test_dataset,model):
    unbatched_dataset = test_dataset.unbatch()

    # Initialize lists to store true labels and predictions
    images = []
    labels = []


    # Iterate over the unbatched dataset to get predictions and true labels
    for img_batch, label_batch in test_dataset:
        images.append(img_batch)
        labels.append(label_batch)

    # Convert lists to numpy arrays
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)

    # Make predictions on the entire dataset
    y_pred = model.predict(images).ravel()  # Flatten predictions if necessary

    # Convert labels to a flat array
    y_true = labels.ravel()
    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return y_true, y_pred


In [8]:
def calculate_weight(pos,neg):

    # Total number of samples
    total = pos + neg

    # Calculate class weights
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0 (Negative): {:.2f}'.format(weight_for_0))
    print('Weight for class 1 (Positive): {:.2f}'.format(weight_for_1))
    return class_weight

In [9]:
# random_choices = random.choices(list(range(df.shape[0])), k=9)
# visualize(df,random_choices)

In [10]:
def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
    labels = df[['target']]

    dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Shuffle and batch the dataset
    if training:
        dataset = dataset.shuffle(len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

from sklearn.metrics import roc_auc_score
import numpy as np

def compute_partial_auc(y_true, y_pred, min_tpr):
    """
    Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

    Parameters:
    y_true (np.array): True binary labels.
    y_pred (np.array): Predicted probabilities or scores.
    min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

    Returns:
    partial_auc (float): The computed partial AUC score.
    """
    # Compute the transformed ground truth and predictions
    v_gt = np.abs(y_true - 1)
    v_pred = 1.0 - y_pred

    # Compute the maximum false positive rate (FPR) based on the minimum TPR
    max_fpr = 1 - min_tpr

    # Compute the partial AUC using roc_auc_score
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    # Rescale the partial AUC
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

    return partial_auc

# Example usage
# y_true = np.array([...])  # Replace with your true labels
# y_pred = np.array([...])  # Replace with your predicted scores
# min_tpr = 0.9  # Example minimum TPR value
# partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# print(f"Partial AUC: {partial_auc:.4f}")


In [11]:
def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
    # Extract labels
    labels = df[['target']]
    
    # Depending on the return type, prepare the dataset accordingly
    if return_type == 'both':
        dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
        dataset = dataset.map(lambda x, y: (load_image_and_metadata(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    elif return_type == 'metadata':
        dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
        dataset = dataset.map(lambda x, y: (preprocess_metadata({k: v for k, v in x.items() if k != 'file_path'}), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    elif return_type == 'images':
        dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
        dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    else:
        raise ValueError("Invalid return_type. Must be one of ['both', 'metadata', 'images']")
    
    # Shuffle and batch the dataset
    if training:
        dataset = dataset.shuffle(len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

from sklearn.metrics import roc_auc_score
import numpy as np


class PartialAUC(tf.keras.metrics.Metric):
    def __init__(self, name='partial_auc', **kwargs):
        super(PartialAUC, self).__init__(name=name, **kwargs)
        self.thresholds = tf.linspace(0.0, 1.0, 101)
        self.tp = self.add_weight(name='tp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fp = self.add_weight(name='fp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fn = self.add_weight(name='fn', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.tn = self.add_weight(name='tn', shape=(101,), initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(tf.cast(y_true, tf.float32), (-1, 1))
        y_pred = tf.cast(y_pred, tf.float32)

        # Debugging shapes
        #tf.print("reshaped y_true shape:", tf.shape(y_true))
        #tf.print("y_pred shape:", tf.shape(y_pred))

        thresholds = self.thresholds[tf.newaxis, :]
        preds = tf.cast(y_pred[:, tf.newaxis] >= thresholds, tf.float32)

        tp = tf.reduce_sum(preds * y_true[:, tf.newaxis], axis=0)
        fp = tf.reduce_sum(preds * (1 - y_true)[:, tf.newaxis], axis=0)
        fn = tf.reduce_sum((1 - preds) * y_true[:, tf.newaxis], axis=0)
        tn = tf.reduce_sum((1 - preds) * (1 - y_true)[:, tf.newaxis], axis=0)

        # Check tensor shapes
        #tf.print("tp shape:", tf.shape(tp))
        #tf.print("fp shape:", tf.shape(fp))
        #tf.print("fn shape:", tf.shape(fn))
        #tf.print("tn shape:", tf.shape(tn))

        # Reshape assuming desired shape is (num_thresholds,)
        num_thresholds = tf.shape(self.thresholds)[0]
        
        reshaped_tp = tf.reshape(tp, [num_thresholds])
        reshaped_fp = tf.reshape(fp, [num_thresholds])
        reshaped_fn = tf.reshape(fn, [num_thresholds])
        reshaped_tn = tf.reshape(tn, [num_thresholds])
        
        self.tp.assign_add(reshaped_tp)
        self.fp.assign_add(reshaped_fp)
        self.fn.assign_add(reshaped_fn)
        self.tn.assign_add(reshaped_tn)

    def result(self):
        # Avoid division by zero
        epsilon = tf.keras.backend.epsilon()
        tpr = self.tp / (self.tp + self.fn + epsilon)
        fpr = self.fp / (self.fp + self.tn + epsilon)

        # Sort fpr and corresponding tpr
        sorted_indices = tf.argsort(fpr)
        sorted_fpr = tf.gather(fpr, sorted_indices)
        sorted_tpr = tf.gather(tpr, sorted_indices)

        # Apply mask for TPR >= 0.8
        mask = sorted_tpr >= 0.8
        partial_fpr = tf.boolean_mask(sorted_fpr, mask)
        partial_tpr = tf.boolean_mask(sorted_tpr, mask)

        # Calculate partial AUC using the trapezoidal rule
        def compute_partial_auc():
            delta_fpr = partial_fpr[1:] - partial_fpr[:-1]
            tpr_sum = partial_tpr[1:] + partial_tpr[:-1]
            return tf.reduce_sum(delta_fpr * tpr_sum / 2)

        partial_auc = tf.cond(tf.shape(partial_fpr)[0] > 1,
                              compute_partial_auc,
                              lambda: tf.constant(0.0))

        return partial_auc * 0.2

    def reset_states(self):
        self.tp.assign(tf.zeros_like(self.tp))
        self.fp.assign(tf.zeros_like(self.fp))
        self.fn.assign(tf.zeros_like(self.fn))
        self.tn.assign(tf.zeros_like(self.tn))
def compute_partial_auc(y_true, y_pred, min_tpr):
    """
    Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

    Parameters:
    y_true (np.array): True binary labels.
    y_pred (np.array): Predicted probabilities or scores.
    min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

    Returns:
    partial_auc (float): The computed partial AUC score.
    """
    # Compute the transformed ground truth and predictions
    v_gt = np.abs(y_true - 1)
    v_pred = 1.0 - y_pred

    # Compute the maximum false positive rate (FPR) based on the minimum TPR
    max_fpr = 1 - min_tpr

    # Compute the partial AUC using roc_auc_score
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    # Rescale the partial AUC
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

    return partial_auc

# Example usage
# y_true = np.array([...])  # Replace with your true labels
# y_pred = np.array([...])  # Replace with your predicted scores
# min_tpr = 0.9  # Example minimum TPR value
# partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# print(f"Partial AUC: {partial_auc:.4f}")


In [12]:
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

clear_memory = ClearMemory()


In [13]:
df_concat = get_sample(df,ratio=20)

positive shape: (393, 55), negative shape: (7860, 55)
Full dataset shape: (8253, 55)


In [14]:
directory = '/kaggle/input/isic-2024-challenge/train-image'
df_concat['file_path'] = df_concat['isic_id'].apply(lambda x: add_path(x, directory))

In [15]:
df_2018 = get_sample(df_2018,ratio = 10)
df_2018['file_path'] = df_2018['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2018-jpg-224x224-resized/train-image'))
df_2019 = get_sample(df_2019,ratio = 10)
df_2019['file_path'] = df_2019['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2019-jpg-224x224-resized/train-image'))
df_2020 = get_sample(df_2020,ratio = 6)
df_2020['file_path'] = df_2020['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2020-jpg-224x224-resized/train-image'))

df_concat_previous = pd.concat([df_2018,df_2019,df_2020],axis=0)

positive shape: (785, 4), negative shape: (7850, 4)
Full dataset shape: (8635, 4)
positive shape: (4522, 4), negative shape: (45220, 4)
Full dataset shape: (49742, 4)
positive shape: (584, 4), negative shape: (3504, 4)
Full dataset shape: (4088, 4)


In [16]:
df_concat_previous.shape

(62465, 5)

In [17]:
df_concat_previous.head()

,Unnamed: 0,isic_id,patient_id,target,file_path
0,8934,ISIC_0036017,dummy_1464,0.0,/kaggle/input/isic-2018-jpg-224x224-resized/tr...
1,3866,ISIC_0029955,HAM_0003906,0.0,/kaggle/input/isic-2018-jpg-224x224-resized/tr...
2,731,ISIC_0024965,HAM_0000733,0.0,/kaggle/input/isic-2018-jpg-224x224-resized/tr...
3,1107,ISIC_0027636,HAM_0001109,0.0,/kaggle/input/isic-2018-jpg-224x224-resized/tr...
4,1274,ISIC_0032374,HAM_0001279,0.0,/kaggle/input/isic-2018-jpg-224x224-resized/tr...


In [18]:
directory = '/kaggle/input/isic-2024-challenge/train-image'
df_concat['file_path'] = df_concat['isic_id'].apply(lambda x: add_path(x, directory))

## Vision Model

In [19]:
positive = df_concat[df_concat['target']==1]

In [20]:

augmented_positive = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
augmented_positive = augmented_positive.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [21]:

augmented_positive1 = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
augmented_positive1 = augmented_positive1.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [22]:
# df_concat['image'] = df_concat['file_path'].apply(preprocess_image)

dataset = tf.data.Dataset.from_tensor_slices((df_concat['file_path'].values, df_concat['target']))
dataset = dataset.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [23]:
previous_dataset = tf.data.Dataset.from_tensor_slices((df_concat_previous['file_path'].values, df_concat_previous['target']))
previous_dataset = previous_dataset.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
previous_dataset = previous_dataset.shuffle(len(previous_dataset))
previous_dataset = previous_dataset.batch(BATCH_SIZE)
previous_dataset = previous_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [24]:
previous_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [25]:
# for image,label in previous_dataset.take(1):
#     print(image.shape,label.shape)
    


In [26]:
dataset = dataset.concatenate(augmented_positive,name='first')
dataset = dataset.concatenate(augmented_positive1,name='version2')

In [27]:
dataset = dataset.shuffle(len(dataset))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [28]:
# efficient.get_layer(name='block4c_add').output

In [29]:
# layer_index = None
# for i, layer in enumerate(efficient.layers):
#     if layer.name == 'block4c_add':
#         layer_index = i
#         break

In [30]:
def create_vision_encoder(trainable_layer,trainable=False,input_shape=(139,139,3)):
    
#     resnet_v2 = keras.applications.ResNet50V2(include_top=False, weights="imagenet", pooling="max")
    model = keras.applications.EfficientNetB0(include_top=True, weights='imagenet',input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3),pooling='avg')
    if trainable:
        for layer in model.layers[:trainable_layer]:
            layer.trainable = trainable
    
    
    
    image = keras.Input(shape=input_shape, name="image")
    
    preprocessed = keras.applications.efficientnet.preprocess_input(image)
    
    
    outputs = model(preprocessed)
#     outputs = model.get_layer(name='block4c_add').output
    outputs = model.layers[118].output
#     outputs = layers.GlobalAveragePooling2D()(outputs)
    return keras.Model([image], outputs, name="vision_encoder")

In [31]:
vision_encoder = create_vision_encoder(trainable_layer=100,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[0],3),trainable=True)
# vision_encoder.summary()

21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [32]:
def vision_model():
    vision_encoder = create_vision_encoder(trainable_layer=100,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[0],3),trainable=True)
#     x = layers.Dense(128,activation='relu')(vision_encoder.output)
#     x = layers.GlobalAveragePooling2D()(vision_encoder.output)
    x = layers.Dense(64,activation='relu')(vision_encoder.output)
    x = layers.Dense(1,activation='sigmoid')(x)

    vision_model = keras.Model(inputs = vision_encoder.input, outputs=x)
    vision_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS)
    return vision_model

In [33]:
model = vision_model()

In [34]:
# model.summary()

In [35]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [36]:
shuffled_dataset = dataset.shuffle(len(dataset))
# num_test_samples = int(len(dataset) * 0.2)  # Assuming 20% of the dataset for testing
# test_dataset = shuffled_dataset.take(num_test_samples)
# train_dataset = shuffled_dataset.take(int(len(dataset) * 0.8))

In [37]:
# len(test_dataset)

In [38]:
# for image, label in dataset.take(1):
#     print(label)

In [39]:
# model = vision_model()
# history = model.fit(dataset, epochs=1)

In [40]:
class_weight = calculate_weight(7860,11790)
class_weight

Weight for class 0 (Negative): 0.83
Weight for class 1 (Positive): 1.25


{0: 0.8333333333333334, 1: 1.25}

In [41]:
# skf = StratifiedKFold(n_splits=5)
# fold_no = 1
# results = []
# images = df_concat[['file_path']]
# target = df_concat[['target']]

# for train_index, val_index in skf.split(images, target):
#     print(f'Fold {fold_no}...')
    
#     X_train, X_val = images.iloc[train_index], images.iloc[val_index]
#     y_train, y_val = target.iloc[train_index], target.iloc[val_index]
    
#     train_df = pd.concat([X_train, y_train],axis=1)
#     test_df = pd.concat([X_val, y_val],axis=1)
#     print(train_df.shape,test_df.shape)
    
#     vision_train_ds = create_dataset(train_df, IMAGE_SIZE, BATCH_SIZE,training=True,return_type='images')
#     vision_test_ds = create_dataset(test_df, IMAGE_SIZE, BATCH_SIZE,return_type='images')
#     # Instantiate the model for each fold
#     model = vision_model()
    
#     # Train the model
#     history = model.fit(vision_train_ds, epochs=2, validation_data=vision_test_ds,class_weight=class_weight)
    
#     # Evaluate the model on the validation set
#     y_true,y_pred = unbatch_test_dataset(vision_test_ds,model)
#     score = compute_partial_auc(y_true,y_pred, min_tpr=0.80)
# #     scores = model.evaluate(vision_test_ds, verbose=0)
#     print(f'Validation auc: {score}')
    
#     results.append(score)
#     fold_no += 1

# # Calculate the average performance
# # average_score = np.mean(results, axis=0)
# # print(f'\nAverage validation loss: {average_score[0]} - Average validation accuracy: {average_score[1]}')


### Metadata Transformations 

In [42]:
df_concat.columns

Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

In [43]:
drop_columns = [
    'isic_id', 'patient_id', 'image_type', 'tbp_tile_type', 'attribution', 
    'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 
    'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm', 
    'tbp_lv_dnn_lesion_confidence'
]
df_concat = df_concat.drop(drop_columns,axis=1)

In [44]:
df_concat.shape

(8253, 40)

In [45]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,60.0,male,anterior torso,4.55,17.039293,14.283063,18.287003,17.740718,24.995039,22.775842,47.022823,51.162436,37.128520,44.167462,9.645337,18.809305,1.080698,2.756229,0.546285,-7.038942,7.267905,6.387014,0.757816,Torso Front Bottom Half,Torso Front,3.075640,2.015081,3.100971,3.050096,13.469302,0.956887,1.629658,1.837165,0.265018,0,-91.055115,1032.430298,-116.544983,/kaggle/input/isic-2024-challenge/train-image/...
1,0,60.0,male,lower extremity,3.20,17.297090,13.842030,18.530230,20.370760,25.348740,24.628640,46.971280,55.803700,20.258580,33.948420,6.267592,13.250990,2.002159,3.455056,-1.840533,-13.689840,14.223060,14.856430,0.654377,Right Leg - Upper,Right Leg,2.328767,98.424420,1.666269,5.285059,9.113276,1.533457,6.230410,2.477048,0.208093,165,142.681100,614.559200,-7.413940,/kaggle/input/isic-2024-challenge/train-image/...
2,0,65.0,male,anterior torso,2.78,25.445058,21.695291,30.435845,27.975466,39.671043,35.402152,50.103610,52.206016,35.701040,40.852079,2.964909,21.321714,0.091567,3.749767,2.460379,-5.151039,5.864022,5.242296,0.887620,Torso Front Top Half,Torso Front,1.456421,0.000403,4.588670,0.202686,7.950908,0.044595,1.005375,2.017189,0.404040,110,-22.539593,1350.984863,-38.696411,/kaggle/input/isic-2024-challenge/train-image/...
3,0,30.0,male,upper extremity,2.66,22.156392,18.991310,38.214378,33.739333,44.172892,38.717082,59.895184,60.625562,54.987482,60.350980,3.039970,19.964219,0.287787,3.165082,4.475045,-5.363498,7.058075,4.985807,0.839318,Left Arm - Upper,Left Arm,1.585048,0.000426,4.246918,0.812312,7.790419,0.254869,0.838690,1.873863,0.391304,20,-387.454468,1380.942871,142.082703,/kaggle/input/isic-2024-challenge/train-image/...
4,1,55.0,male,upper extremity,2.11,31.787660,25.694930,33.371060,34.688920,46.087770,43.168860,46.392050,53.471750,35.406070,45.280460,2.758491,13.757490,0.562755,6.092734,-1.317865,-9.874387,10.160410,8.401830,0.668058,Right Arm - Upper,Right Arm,1.580169,76.485220,1.823582,1.517963,6.160351,0.452263,3.570706,2.064051,0.216867,45,313.384500,1511.212000,121.424400,/kaggle/input/isic-2024-challenge/train-image/...


In [46]:
df_concat.dtypes

target                           int64
age_approx                     float64
sex                             object
anatom_site_general             object
clin_size_long_diam_mm         float64
tbp_lv_A                       float64
tbp_lv_Aext                    float64
tbp_lv_B                       float64
tbp_lv_Bext                    float64
tbp_lv_C                       float64
tbp_lv_Cext                    float64
tbp_lv_H                       float64
tbp_lv_Hext                    float64
tbp_lv_L                       float64
tbp_lv_Lext                    float64
tbp_lv_areaMM2                 float64
tbp_lv_area_perim_ratio        float64
tbp_lv_color_std_mean          float64
tbp_lv_deltaA                  float64
tbp_lv_deltaB                  float64
tbp_lv_deltaL                  float64
tbp_lv_deltaLB                 float64
tbp_lv_deltaLBnorm             float64
tbp_lv_eccentricity            float64
tbp_lv_location                 object
tbp_lv_location_simple   

In [47]:
df_concat  = encode(df_concat,['sex','anatom_site_general','tbp_lv_location','tbp_lv_location_simple'])

In [48]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,60.0,1,0,4.55,17.039293,14.283063,18.287003,17.740718,24.995039,22.775842,47.022823,51.162436,37.128520,44.167462,9.645337,18.809305,1.080698,2.756229,0.546285,-7.038942,7.267905,6.387014,0.757816,17,6,3.075640,2.015081,3.100971,3.050096,13.469302,0.956887,1.629658,1.837165,0.265018,0,-91.055115,1032.430298,-116.544983,/kaggle/input/isic-2024-challenge/train-image/...
1,0,60.0,1,2,3.20,17.297090,13.842030,18.530230,20.370760,25.348740,24.628640,46.971280,55.803700,20.258580,33.948420,6.267592,13.250990,2.002159,3.455056,-1.840533,-13.689840,14.223060,14.856430,0.654377,12,4,2.328767,98.424420,1.666269,5.285059,9.113276,1.533457,6.230410,2.477048,0.208093,165,142.681100,614.559200,-7.413940,/kaggle/input/isic-2024-challenge/train-image/...
2,0,65.0,1,0,2.78,25.445058,21.695291,30.435845,27.975466,39.671043,35.402152,50.103610,52.206016,35.701040,40.852079,2.964909,21.321714,0.091567,3.749767,2.460379,-5.151039,5.864022,5.242296,0.887620,18,6,1.456421,0.000403,4.588670,0.202686,7.950908,0.044595,1.005375,2.017189,0.404040,110,-22.539593,1350.984863,-38.696411,/kaggle/input/isic-2024-challenge/train-image/...
3,0,30.0,1,4,2.66,22.156392,18.991310,38.214378,33.739333,44.172892,38.717082,59.895184,60.625562,54.987482,60.350980,3.039970,19.964219,0.287787,3.165082,4.475045,-5.363498,7.058075,4.985807,0.839318,3,1,1.585048,0.000426,4.246918,0.812312,7.790419,0.254869,0.838690,1.873863,0.391304,20,-387.454468,1380.942871,142.082703,/kaggle/input/isic-2024-challenge/train-image/...
4,1,55.0,1,4,2.11,31.787660,25.694930,33.371060,34.688920,46.087770,43.168860,46.392050,53.471750,35.406070,45.280460,2.758491,13.757490,0.562755,6.092734,-1.317865,-9.874387,10.160410,8.401830,0.668058,9,3,1.580169,76.485220,1.823582,1.517963,6.160351,0.452263,3.570706,2.064051,0.216867,45,313.384500,1511.212000,121.424400,/kaggle/input/isic-2024-challenge/train-image/...


In [49]:
df_concat.columns

Index(['target', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'file_path'],
      dtype='object')

In [50]:
columns = ['age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z']

In [51]:
df_concat = scale_columns(df_concat,columns)

In [52]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,0.148115,0.557359,-1.584460,0.281186,-0.719557,-0.193560,-1.825143,-1.998549,-1.666486,-1.659158,-1.273984,-1.628760,-0.468864,-0.585625,0.042815,-0.056022,-0.026933,-0.871538,-0.318074,0.547263,-0.634359,-0.475747,0.121507,0.995758,1.100169,0.365204,-0.875425,-0.208529,-0.053840,0.196558,-0.111844,-0.629245,-0.629681,-0.337447,-1.643822,-0.422911,-0.014564,-1.946918,/kaggle/input/isic-2024-challenge/train-image/...
1,0,0.148115,0.557359,-0.154469,-0.418759,-0.657341,-0.313735,-1.779863,-1.419253,-1.605582,-1.280150,-1.282878,-0.843999,-2.015858,-1.434365,-0.254650,-1.086258,1.093430,-0.603368,-1.357617,-1.330737,1.320934,2.846799,-0.606196,0.084794,0.004514,-0.207093,1.445756,-1.040879,0.985581,-0.461528,0.622578,1.939138,0.324147,-0.791434,1.482484,0.775065,-1.007958,-0.714289,/kaggle/input/isic-2024-challenge/train-image/...
2,0,0.515668,0.557359,-1.584460,-0.636520,1.309095,1.826162,0.436557,0.255767,0.860606,0.923673,-0.742394,-1.452308,-0.599766,-0.860983,-0.545505,0.409654,-1.229574,-0.490275,0.515581,1.080346,-1.029031,-0.924819,1.034700,1.177951,1.100169,-0.875534,-0.923931,0.654567,-1.378093,-0.637133,-1.273900,-0.977753,-0.361332,0.771282,0.440382,-0.071746,0.742728,-1.067623,/kaggle/input/isic-2024-challenge/train-image/...
3,0,-2.057202,0.557359,1.275521,-0.698737,0.515406,1.089368,1.884655,1.525323,1.635788,1.601773,0.947141,-0.028702,1.168824,0.758493,-0.538895,0.158041,-0.990999,-0.714644,1.393039,1.020354,-0.693349,-1.025440,0.694889,-1.554942,-1.638969,-0.776972,-0.923930,0.456298,-1.094573,-0.661379,-1.006059,-1.070806,-0.574978,0.669710,-1.264876,-1.942057,0.813946,0.974267,/kaggle/input/isic-2024-challenge/train-image/...
4,1,-0.219438,0.557359,1.275521,-0.983900,2.839823,2.916004,0.982994,1.734480,1.965515,2.512426,-1.382824,-1.238293,-0.626815,-0.493185,-0.563683,-0.992378,-0.656677,0.408821,-1.129977,-0.253376,0.178807,0.314664,-0.509952,-0.461785,-0.543314,-0.780711,0.917541,-0.949613,-0.766394,-0.907641,-0.754622,0.454351,-0.291478,-0.721452,-0.791193,1.649977,1.123631,0.740933,/kaggle/input/isic-2024-challenge/train-image/...


In [53]:
# train_df, test_df = train_test_split(df_concat, test_size=0.11, 
#                                      stratify=df_concat["target"].values, random_state=42)
# # 5% for validation
# # train_df, val_df = train_test_split(train_df, test_size=0.11,stratify=train_df["target"].values, random_state=42)

# print(f"Total training examples: {len(train_df)}")
# # print(f"Total validation examples: {len(val_df)}")
# print(f"Total test examples: {len(test_df)}")

In [54]:
# labels = df['target']

In [55]:
# labels.shape

https://github.com/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment.ipynb

In [56]:
class ProjectEmbeddings(keras.layers.Layer):
    def __init__(self, num_projection_layers, projection_dims, dropout_rate, **kwargs):
        super(ProjectEmbeddings, self).__init__(**kwargs)
        self.num_projection_layers = num_projection_layers
        self.projection_dims = projection_dims
        self.dropout_rate = dropout_rate
        
        # Initialize Dense and Dropout layers in __init__ or build
        self.projection_layers = [layers.Dense(self.projection_dims) for _ in range(self.num_projection_layers)]
        self.dropout_layers = [layers.Dropout(self.dropout_rate) for _ in range(self.num_projection_layers - 1)]

    def call(self, embeddings):
        x = embeddings
        for i in range(self.num_projection_layers):
            x = self.projection_layers[i](x)
            if i < self.num_projection_layers - 1:
                x = tf.nn.gelu(x)
                x = self.dropout_layers[i](x)
        return x

In [57]:
df_concat.shape

(8253, 40)

(7860, 56), negative shape: (11790, 56)

In [58]:
# vision_test_ds

In [59]:
# # Create the dataset
# vision_train_ds = create_dataset(train_df, IMAGE_SIZE, batch_size,training=True,return_type='images')
# vision_test_ds = create_dataset(test_df, IMAGE_SIZE, batch_size,return_type='images')
# # val_ds = create_dataset(val_df, image_size, batch_size)

In [60]:
# vision_history = vision_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=20,
#     class_weight=class_weight,
#     callbacks=[partial_auc_callback]
# )

In [61]:
# for (images, metadata), labels in train_ds.take(1):
#     print(images.shape,metadata.shape,labels.shape)

In [62]:
# !pip install -q focal-loss

In [63]:
# from focal_loss import BinaryFocalLoss

In [64]:
# def create_table_encoder(
#     num_projection_layers, projection_dims, dropout_rate, trainable=False
# ):
#     table_input = tf.keras.Input(shape=(38,))
#     dense_1024 = tf.keras.layers.Dense(1024, activation='relu')(table_input)
#     dense_512 = tf.keras.layers.Dense(512, activation='relu')(dense_1024)

#     # Project the embeddings produced by the model.
#     outputs = ProjectEmbeddings(num_projection_layers, projection_dims, dropout_rate)(dense_512)

#     # Create the text encoder model.
#     return keras.Model(table_input, outputs, name="table_encoder")

In [65]:
# table_encoder = create_table_encoder(num_projection_layers=1, projection_dims=256, dropout_rate=0.1)
# # keras.utils.plot_model(table_encoder, show_shapes=True)

In [66]:
# def create_multimodal_model(num_projection_layers=1, projection_dims=256, dropout_rate=0.1, 
#                      vision_trainable=False, text_trainable=False):
#     # Receive the images as inputs.
#     image_1 = keras.Input(shape=(139, 139, 3), name="image_1")
#     table_input = tf.keras.Input(shape=(38,))

    

#     # Create the encoders.
#     vision_encoder = create_vision_encoder(num_projection_layers, projection_dims, dropout_rate, vision_trainable)
#     table_encoder = create_table_encoder(num_projection_layers, projection_dims, dropout_rate, text_trainable)

#     # Fetch the embedding projections.
#     vision_projections = vision_encoder(image_1)
#     table_projections = table_encoder(table_input)

#     # Concatenate the projections and pass through the classification layer.
#     concatenated = keras.layers.Concatenate()([vision_projections, table_projections])
#     outputs = keras.layers.Dense(1, activation="sigmoid")(concatenated)
#     return keras.Model([image_1, table_input], outputs)

In [67]:
# vision_model = create_vision_encoder(2, 256, 0.5,True)
# keras.utils.plot_model(vision_model, show_shapes=True)

In [68]:
# vision_model.output

In [69]:

# multimodal_model = create_multimodal_model(vision_trainable=True, text_trainable=True)
# keras.utils.plot_model(multimodal_model, show_shapes=True)

In [70]:
# multimodal_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS)


In [71]:
# history = multimodal_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10,
#     class_weight=class_weight
# )

In [72]:
# from focal_loss import BinaryFocalLoss

In [73]:
# multimodal_model_focal = create_multimodal_model()
# multimodal_model_focal.compile(
#     optimizer="adam", loss=BinaryFocalLoss(gamma=2), metrics=METRICS
# )
# history = multimodal_model_focal.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10
# )